In [8]:
import pandas as pd
import requests
import numpy as np

In [9]:
street = pd.read_csv('tickets_update.csv', error_bad_lines=False)
street = street.replace(np.nan, '', regex=True)

In [10]:
df3 = street.iloc[50001:75000]

In [11]:
df3.head()

,street_name,borough
50001,STELL,Brooklyn
50002,N/S CORNELIA ST,Staten Island
50003,W/S MALCOLM BLVD X,Brooklyn
50004,W/S OF 23 ST SOFT OF,Queens
50005,S/E/C 144 ST,Queens


In [12]:
API_KEY = ''

In [13]:
addresses = (df3['street_name'] + ', ' + df3['borough'] + ', NY,US').tolist()
def get_google_results(address, api_key=None, return_full_response=False):
    # Set up Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address" : None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            #"google_place_id": None,
            "type": None,
            #"postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "accuracy": answer.get('geometry').get('location_type'),
            #"google_place_id": answer.get("place_id"),
            "type": ",".join(answer.get('types')),
            #"postcode": ",".join([x['long_name'] for x in answer.get('address_components') 
            #                      if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['input_string'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')
    if return_full_response is True:
        output['response'] = results
    
    return output

In [14]:
results = []

In [15]:
%%time
# Create a list to hold results
#results = []
# Go through each address in turn
for address in addresses:
    geocode_result = get_google_results(address, API_KEY)
    results.append(geocode_result)           

result3 = pd.DataFrame(results)

CPU times: user 4min 46s, sys: 6.89 s, total: 4min 53s
Wall time: 4h 3min 26s


In [16]:
result3.head()

,accuracy,formatted_address,input_string,latitude,longitude,number_of_results,status,type
0,ROOFTOP,"319 Frost St, Brooklyn, NY 11222, USA","STELL, Brooklyn, NY,US",40.719358,-73.938794,1,OK,"establishment,point_of_interest"
1,GEOMETRIC_CENTER,"Cornelia St, Staten Island, NY 10304, USA","N/S CORNELIA ST, Staten Island, NY,US",40.595112,-74.091275,1,OK,route
2,GEOMETRIC_CENTER,"Malcolm X Blvd, Brooklyn, NY, USA","W/S MALCOLM BLVD X, Brooklyn, NY,US",40.687100,-73.929888,1,OK,route
3,GEOMETRIC_CENTER,"W 23rd St, New York, NY, USA","W/S OF 23 ST SOFT OF, Queens, NY,US",40.745306,-73.998475,1,OK,route
4,GEOMETRIC_CENTER,"144th St, Queens, NY 11435, USA","S/E/C 144 ST, Queens, NY,US",40.706704,-73.812115,1,OK,route


In [17]:
result3.tail()

,accuracy,formatted_address,input_string,latitude,longitude,number_of_results,status,type
24994,GEOMETRIC_CENTER,"Jay Ave, Queens, NY 11378, USA","N/S JAY AV, Queens, NY,US",40.727530,-73.898691,1,OK,route
24995,GEOMETRIC_CENTER,"Patchen Ave, Brooklyn, NY, USA","PATCHEN ST, Brooklyn, NY,US",40.685972,-73.926671,1,OK,route
24996,GEOMETRIC_CENTER,"Grand Concourse, The Bronx, NY, USA","L/S GRAND CONCOURSE, , NY,US",40.847038,-73.908254,1,OK,route
24997,GEOMETRIC_CENTER,"E 166th St, The Bronx, NY, USA","E166TH ST, Bronx, NY,US",40.828787,-73.910298,1,OK,route
24998,GEOMETRIC_CENTER,"W 168th St, The Bronx, NY, USA","W 168 TH ST, Bronx, NY,US",40.838357,-73.924934,1,OK,route


In [18]:
result3.to_csv('3.csv')

In [19]:
df3.reset_index(inplace=True)

In [20]:
new = pd.merge(df3, result3, left_index=True, right_index=True)

In [21]:
new.head()

,index,street_name,borough,accuracy,formatted_address,input_string,latitude,longitude,number_of_results,status,type
0,50001,STELL,Brooklyn,ROOFTOP,"319 Frost St, Brooklyn, NY 11222, USA","STELL, Brooklyn, NY,US",40.719358,-73.938794,1,OK,"establishment,point_of_interest"
1,50002,N/S CORNELIA ST,Staten Island,GEOMETRIC_CENTER,"Cornelia St, Staten Island, NY 10304, USA","N/S CORNELIA ST, Staten Island, NY,US",40.595112,-74.091275,1,OK,route
2,50003,W/S MALCOLM BLVD X,Brooklyn,GEOMETRIC_CENTER,"Malcolm X Blvd, Brooklyn, NY, USA","W/S MALCOLM BLVD X, Brooklyn, NY,US",40.687100,-73.929888,1,OK,route
3,50004,W/S OF 23 ST SOFT OF,Queens,GEOMETRIC_CENTER,"W 23rd St, New York, NY, USA","W/S OF 23 ST SOFT OF, Queens, NY,US",40.745306,-73.998475,1,OK,route
4,50005,S/E/C 144 ST,Queens,GEOMETRIC_CENTER,"144th St, Queens, NY 11435, USA","S/E/C 144 ST, Queens, NY,US",40.706704,-73.812115,1,OK,route


In [22]:
new.to_csv('3.csv')